<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Get database
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Notion/Notion_Get_database.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import library

In [1]:
import requests
import pandas as pd
import json

### Variables

In [2]:
# Enter Notion Token API
TOKEN_API = 'secret_R1CrUGn8bx9itbJW0Fc9Cc0R9Lmhbnz2ayqEe0GhRPq'

# Enter database url
DATABASE_URL = 'https://www.notion.so/d0bb915c4cb4422a87679f3bb9658282?v=8cd7c6ea0ec244da9eb65aa88a87aabb'

# Set notion version
_VERSION = '2021-08-16'

## Model
### Request the database
1. create the header
2. parse the database id from URL
3. Request the Notion API

In [3]:
def create_headers(token_api, version):
    return {
            'Authorization': f'Bearer {token_api}',
            'Notion-Version': f'{version}',
            'Content-Type': 'application/json'
        }

create_headers(TOKEN_API, _VERSION)

In [4]:
def get_id_from_url(database_url):
    path = database_url.split('/')[-1]
    id = path.split('?')[0]
    return id

get_id_from_url(DATABASE_URL)

In [5]:
# make a request to Notion API and receive a Json 
def fetch_raw_data(token_api, database_url):
    database_id = get_id_from_url(database_url)
    url = f'https://api.notion.com/v1/databases/{database_id}/query'
    headers = create_headers(token_api, _VERSION)
    response = requests.post(url, headers=headers)
    try:
        response.raise_for_status()
    except requests.HTTPError as e:
        return e
    return response.json()

pages_json = fetch_raw_data(TOKEN_API, DATABASE_URL)
pages_df = pd.DataFrame(pages_json['results'])
pages_df

### Data preparation
As we can see the content of the Notion table is in the properties column. We will now extract it and see what it contains. The column properties contain a dictionary for each Notion record. We will exctract each of these disctionnaries and create a new dataFrame.

In [6]:
def unstack_properties(df):
    list_dict = [row['properties'] for _, row in df.iterrows()]
    return pd.DataFrame(list_dict)

df_properties = unstack_properties(pages_df)
df_properties.head()

As we can see, each of the properties contains another dict of the information.<br>
🚨 the usefull information is different depending on the type of the properties <br>
👉 [more info](https://developers.notion.com/reference/page#all-property-values)

Let's create a function to extract the usefull information of each properties

In [7]:
def extract_text(dictionnary):
    if 'name' in dictionnary:
        return dictionnary['name']
    elif 'plain_text' in dictionnary:
        return dictionnary['plain_text']
    else:
        return ''

def extract_date(dictionnary):
    '''
    For the moment we extract only the starting date of a date field
    Example {'id': 'prop_1', 'type': 'date', 'date': {'start': '2018-03-21', 'end': None}}
    '''
    return dictionnary['start']
    
def extract_data(element):
    ''' 
    input: a dictionnary of a notion property
    Exemple: {'id': 'W#4k', 'type': 'select', 'select': {'id': 'b305bd26-****-****-****-c78e2034db8f', 'name': 'Client', 'color': 'green'}}
    output: the string containing the information of the dict. (Client in the exemple)
    '''
    if type(element) is dict:
        dict_type = element['type'] 
        informations = element[dict_type]

        if type(informations) is dict:
            if dict_type == 'date':
                return extract_date(informations)
            else:
                return extract_text(informations)
        
        elif type(informations) is list:
            informations = [extract_text(elm) for elm in informations]
            return ','.join(informations)
        else:
            return informations
    else:
        return ''

In [8]:
clean_properties = df_properties.applymap(extract_data)
clean_properties

delete some useless columns and set the `Name` of the page as the index

In [9]:
final_df = pd.concat([clean_properties, pages_df], axis=1)
final_df.drop(['parent', 'properties'], axis=1, inplace=True)
final_df.set_index("Name", inplace=True)

### Group everything together

In [16]:
def get_database(TOKEN_API, DATABASE_URL):
    raw_data = fetch_raw_data(TOKEN_API, DATABASE_URL)
    raw_df = pd.DataFrame(raw_data['results'])

    properties = unstack_properties(raw_df)
    clean_properties = properties.applymap(extract_data)
    full_df = pd.concat([clean_properties, pages_df], axis=1)
    full_df.drop(['parent', 'properties'], axis=1, inplace=True)
    full_df.columns = full_df.columns.str.upper()
    return full_df

## Output

In [17]:
get_database(TOKEN_API, DATABASE_URL)